En Python existe una propuesta de API est�ndar para el manejo de bases de datos, de forma que el c�digo sea pr�cticamente igual independientemente de la base de datos que se est� utilizando por debajo. Esta especificaci�n recibe el nombre de Python Database API o DB-API.

Se van a estudiar dos casos:

* SQLite.

* MySQL


### SQLite

Python tiene integrada una base de datos relacional denominada SQLite.

Para crear una base de datos:

* En primer lugar hay que realizar una conexi�n con el servidor de la base de datos. Esto se hace mediante la funci�n connect, cuyos par�metros no est�n estandarizados y dependen de la base de datos. 

* En el caso de sqlite3 s�lo se necesita pasar como par�metro una cadena con la ruta al archivo en el que guardar los datos de la base de datos, o bien la cadena �:memory:� para utilizar la memoria RAM en lugar de un fichero en disco.

Vamos a crear una base de datos denominada �Biblioteca�

In [1]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")

La funci�n connect devuelve un objeto de tipo Connection que representa la conexi�n con conexi�n con la base de datos almacenada en el archivo biblioteca.sqlite3 del directorio actual. Si el archivo no existe, se crear� nuevo. 

Las distintas operaciones que se pueden realizar con la base de datos se realizan a trav�s de un objeto Cursor. Para crear este objeto se utiliza el m�todo cursor() del objeto Connection.


In [4]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")
cur=conn.cursor()

Una vez que tenemos el cursor, se pueden ejecutar comandos sobre el contenido de la base de datos, usando el m�todo execute() que toma como argumento una cadena con el c�digo SQL a ejecutar.

Por ejemplo se va a crear una tabla llamada Libros con una columna de texto llamada �T�tulo� y otra columna de enteros llamada �prestamos�. Adem�s antes de crear la tabla la vamos a eliminar para asegurarse que no existe ya en la base de datos.

In [10]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")
cur=conn.cursor()
cur.execute("DROP TABLE IF EXISTS Libros")
cur.execute("CREATE TABLE Libros (titulo TEXT, ejemplares INTEGER)")
cur.close()

Una vez creada la tabla Libros se pueden guardar datos usando una llamada a execute() con el comando SQL INSERT. Este comando indica qu� tabla se va a utilizar y luego define una fila nueva, enumerando los campos a incluir y seguidos por los valores (VALUES) que se desean colocar en esa fila. 


In [13]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")
cur=conn.cursor()
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES ('El Quijote',20)")
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES ('El Escarabajo de oro', 15)")
cur.close()

Otra forma de insertar consiste en especificar como signos de interrogaci�n  (?,?) los valores para indicar que ser�n pasados como una tupla en el segundo par�metro de la llamada a execute().


In [15]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")
cur=conn.cursor()
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)", ('El Quijote',20))
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)",  ('El Escarabajo de oro', 15))
cur.close()

Si la base de datos soporta transacciones y est�n activadas, y adem�s la caracter�stica de auto-commit est� desactivada, ser� necesario llamar al m�todo commit de la conexion para que se lleven a cabo las operaciones definidas en la transacci�n.

Si en estas circunstancias se usara una herramienta externa para comprobar el contenido de la base de datos sin hacer primero el commit aparecer�a entonces con una base de datos vac�a. Sin embargo si se consulta desde Python parecer�a que se han llevado a cabo los cambios, aunque no es as�. 

In [16]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")
cur=conn.cursor()
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)", ('El Quijote',20))
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)",  ('El Escarabajo de oro', 15))
cur.close()
conn.commit()

En el ejemplo primero se insertan dos filas en la tabla con INSERT y luego se usa commit() para forzar que los datos sean escritos en el archivo de la base de datos.

Si la  base de datos soporta la caracter�stica de rollback  entonces se puede cancelar la transacci�n actual con el m�todo rollback de la conexi�n. Si la base de datos no soporta rollback, entonces al llamar a este m�todo producir� una excepci�n.

Cuando se quieren insertar m�ltiples filas en una sola operaci�n se puede usar el m�todo executemany y proporcionar como argumento la secuencia de filas que se quieren insertar como una lista. Como resultado se llama al m�todo execute una vez por cada fila.

In [17]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")
cur=conn.cursor()
cur.executemany("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)", [('El Quijote',20),('El Escarabajo de oro', 15)])
print (cur.rowcount)
cur.close()
conn.commit()

2


El mismo efecto se podr�a haber conseguido utilizando un bucle sobre una lista y el m�todo execute().

In [19]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")
cur=conn.cursor()
entradas=[('El Quijote',20),('El Escarabajo de oro', 15)]
for fila in entradas:
    cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)", fila)
cur.close()
conn.commit()

Para realizar consultas a la base de datos tambi�n se utiliza el m�todo execute tomando como argumento una cadena que represente una sentencia SELECT de SQL.

Cuando se realiza una consulta, el cursor no lee todos los datos de la base de datos cuando se ejecuta la sentencia SELECT sino que los datos ser�n le�dos a medida que se pidan las filas.

Para consultar las tuplas resultantes de la sentencia SQL se puede llamar a los m�todos de cursor __fetchone, fetchmany o fetchall__ o usar el objeto cursor como un iterador.

En los siguientes ejemplos primero se insertaran dos filas en la tabla con INSERT y luego se usar� commit() para forzar que los datos sean escritos en el archivo de la base de datos. Despu�s se usar� el comando SELECT para recuperar las filas que se acaban de insertar en la tabla, y en cada ejemplo se usar� una forma distinta de pedir las filas recuperadas. Al final del programa se  ejecuta el comando DELETE para borrar las filas que se acaban de crear, y por �ltimo se ejecuta un commit() para forzar a los datos a ser eliminados de la base de datos.

En este ejemplo se usa fetchall que recupera una lista de las filas que hay en la tabla.

In [21]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")
cur=conn.cursor()
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)", ('El Quijote',20))
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)",  ('El Escarabajo de oro', 15))
conn.commit()
print ("Libros")
cur.execute("SELECT titulo, ejemplares FROM Libros")
print (cur.fetchall())
cur.execute("DELETE FROM Libros")
cur.close()
conn.commit()


Libros
[('El Quijote', 20), ('El Escarabajo de oro', 15)]


Tambi�n es posible iterar sobre las tuplas obtenidas con fetchall:

In [1]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")
cur=conn.cursor()
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)", ('El Quijote',20))
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)",  ('El Escarabajo de oro', 15))
conn.commit()
print ("Libros")
cur.execute("SELECT titulo, ejemplares FROM Libros")
for fila in cur.fetchall():
    print (fila)
cur.execute("DELETE FROM Libros")
cur.close()
conn.commit()

Libros
('El Quijote', 20)
('El Escarabajo de oro', 15)


Tambi�n es posible recuperar los valores de las tuplas utilizando tuplas en el bucle.

In [2]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")
cur=conn.cursor()
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)", ('El Quijote',20))
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)",  ('El Escarabajo de oro', 15))
conn.commit()
print ("Libros")
cur.execute("SELECT titulo, ejemplares FROM Libros")
for (titulo,ejemplar) in cur.fetchall():
    print ("T�tulo:",titulo)
cur.execute("DELETE FROM Libros")
cur.close()
conn.commit()

Libros
T�tulo: El Quijote
T�tulo: El Escarabajo de oro


En el siguiente ejemplo se recupera una �nica columna de la tabla y luego se usa fetchall:

In [7]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")
cur=conn.cursor()
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)", ('El Quijote',20))
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)",  ('El Escarabajo de oro', 15))
conn.commit()
print ("Libros")
cur.execute("SELECT titulo FROM Libros")
titulos= cur.fetchall()
print (titulos)
cur.execute("DELETE FROM Libros")
cur.close()
conn.commit()

Libros
[('El Quijote',), ('El Escarabajo de oro',)]


En este �ltimo caso se podr�a formatear para que en vez de devolver una tupla con un solo valor devolviera una lista con los valores recuperados usando comprensi�n de listas

In [8]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")
cur=conn.cursor()
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)", ('El Quijote',20))
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)",  ('El Escarabajo de oro', 15))
conn.commit()
print ("Libros")
cur.execute("SELECT * FROM Libros")
titulos= [rec[0] for rec in cur.fetchall()]
print (titulos)
cur.execute("DELETE FROM Libros")
cur.close()
conn.commit()

Libros
['El Quijote', 'El Escarabajo de oro']


Si las tablas que se usan son muy grandes es mejor no pedir todas las filas mediante fetchall, usando otros m�todos alternativos como fetchone que devuelve la siguiente tupla del conjunto resultado o None si no existen m�s 

In [9]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")
cur=conn.cursor()
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)", ('El Quijote',20))
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)",  ('El Escarabajo de oro', 15))
conn.commit()
print ("Libros")
cur.execute("SELECT * FROM Libros")
while True:
    fila=cur.fetchone()
    if not fila: break
    print (fila)
cur.execute("DELETE FROM Libros")
cur.close()
conn.commit()

Libros
('El Quijote', 20)
('El Escarabajo de oro', 15)


Otra alternativa a fetchall es el m�todo fetchmany que devuelve el n�mero de tuplas indicado por el entero pasado como par�metro o bien el n�mero indicado por el atributo Cursor.arraysize si no se pasa ning�n par�metro( por defecto vale 1).

In [10]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")
cur=conn.cursor()
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)", ('El Quijote',20))
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)",  ('El Escarabajo de oro', 15))
conn.commit()
print ("Libros")
cur.execute("SELECT * FROM Libros")
while True:
    filas=cur.fetchmany()
    if not filas: break
    for fila in filas:
        print(fila)
cur.execute("DELETE FROM Libros")
cur.close()
conn.commit()

Libros
('El Quijote', 20)
('El Escarabajo de oro', 15)


Observar que una vez que se han recuperado todas las filas con fetchall, fetchone o fetchmany, si se quieren volver a recuperar las filas ser�a necesario realizar una nueva llamada a execute con la sentencia SELECT dado que se pierden una vez recuperadas.

Alternativamente a los m�todos anteriores, tambi�n es posible iterar sobre el cursor con el que se ha realizado la consulta.

In [11]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")
cur=conn.cursor()
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)", ('El Quijote',20))
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)",  ('El Escarabajo de oro', 15))
conn.commit()
print ("Libros")
cur.execute("SELECT * FROM Libros")
for resultado in cur:
    print (resultado)
cur.execute("DELETE FROM Libros")
cur.close()
conn.commit()

Libros
('El Quijote', 20)
('El Escarabajo de oro', 15)


Para realizar actualizaciones o borrados tambi�n se usa el m�todo execute del objeto cursor. En el siguiente ejemplo se va actualizar la columna �ejemplares� de la fila correspondiente al libro con t�tulo �El quijote� y se va a rellenar con el valor 22.

Si se hace la consulta para ver lo que hay en la base de datos:

In [19]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")
cur=conn.cursor()
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)", ('El Quijote',20))
cur.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)",  ('El Escarabajo de oro', 15))
conn.commit()
print ("Libros")
cur.execute("SELECT * FROM Libros")
for (titulo, ejemplar) in cur.fetchall():
    print ("Titulo:", titulo)
    print ("Ejemplar:", ejemplar)
cur.execute("DELETE FROM Libros")
cur.close()
conn.commit()

Libros
Titulo: El Quijote
Ejemplar: 20
Titulo: El Escarabajo de oro
Ejemplar: 15


Ahora se realiza la actualizaci�n:

In [21]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")
cur=conn.cursor()
cur.execute('UPDATE Libros set ejemplares=? WHERE titulo=?',[22,"El Quijote"])
conn.commit()
cur.execute("SELECT * FROM Libros")
for (titulo, ejemplar) in cur.fetchall():
    print ("Titulo:", titulo)
    print ("Ejemplar:", ejemplar)
cur.close()
conn.commit()

Titulo: El Quijote
Ejemplar: 22
Titulo: El Escarabajo de oro
Ejemplar: 15


En el siguiente ejemplo se va eliminar la fila correspondiente al libro con t�tulo �El quijote�.

In [22]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")
cur=conn.cursor()
cur.execute('DELETE FROM Libros WHERE titulo=?',["El Quijote"])
conn.commit()
cur.execute("SELECT * FROM Libros")
for (titulo, ejemplar) in cur.fetchall():
    print ("Titulo:", titulo)
    print ("Ejemplar:", ejemplar)
cur.close()
conn.commit()

Titulo: El Escarabajo de oro
Ejemplar: 15


Se pueden realizar operaciones de columna cuando se ejecuta una sentencia SELECT. En el siguiente ejemplo se van a sumar los ejemplares de todos los libros almacenados en la base de datos.

In [25]:
import sqlite3
conn=sqlite3.connect("biblioteca.sqlite3")
cur=conn.cursor()
cur.execute('SELECT sum(ejemplares) from Libros')
print (cur.fetchall())
cur.close()
conn.commit()

[(15,)]


Se quiere implementar un programa que guarde informaci�n de los amigos que una persona tiene en Twitter y de las relaciones que tienen a su vez estos amigos entre s�. Esta informaci�n se almacenar� en una base de datos.

En primer lugar se fija la estructura de la base de datos que se va a utilizar:

* Se crear� una tabla llamada Personas que almacenar� la informaci�n de las cuentas de Twitter.

* Se crear� una tabla llamada Seguimientos que almacenar� las relaciones que existen entre las personas.

La tabla Personas dispondr� de 3 columnas:

* Un id que actuar� como clave primar�a de la tabla.

* El nombre de usuario de la cuenta de Twitter.

* Un valor entero que puede valer 0 o 1 que indica si la informaci�n de dicha cuenta ha sido recuperada o no.


La tabla Seguimiento dispondr� de 2 columnas que contendr�n id�s de usuarios registrados en la tabla Personas y que representar�n una relaci�n de seguimiento entre ambos usuarios con un sentido definido.

Observar que la combinaci�n de los dos n�meros de cada fila de la tabla Seguimientos se especifica como �nica para evitar que se cometan errores como a�adir la misma relaci�n entre las mismas personas m�s de una vez.

Estas restricciones obligan a que las inserciones que se hagan en las tablas a�adan la clausula IGNORE en la sentencia INSERT para indicar que si este INSERT en concreto causara una violaci�n de la regla �el nombre debe ser �unico�, el sistema de la base de datos est� autorizado a ignorar el INSERT.

La estructura que se va a seguir para implementar el programa ser�:

* Crear tablas con claves primarias y restricciones.

* Cuando se disponga de una cuenta, y se necesita el valor del id de esa persona, dependiendo de si esa persona ya est� o no  en la tabla Personas: 

   * Buscar lapersona en la tabla Personas y recuperar el valor de id para esa persona, 

   * A�adir la persona a la tabla Personas y obtener el valor del id para la fila reci�n a�adida.

   * Insertar la fila que indica la relaci�n de �seguimiento�.
   
En el siguiente trozo de c�digo se crean las tablas.


En el siguiente trozo de c�digo se pide al usuario una cuenta de Twitter, de forma que si la cuenta ya existe se debe averiguar el valor de su id, y si la cuenta no existe a�n en la tabla Personas, se debe insertar el registro y obtener el valor del id de la fila reci�n insertada(para ello se us cur.lastrowid que proporciona el valorque la base de datos ha asignado a la columna id en la fila reci�n creada)


A continuaci�n se recupera la informaci�n de amistades del usuario y se actualiza la tabla Personas para indicar que esa cuenta ya ha sido procesada(para lo cual se fija el campo recuperado a 1).

El siguiente trozo de c�digo realiza la b�squeda de las amistades del usuario en el documento json recuperado desde Twitter.

Una vez extraido el nombre de usuario del documento JSON, se intenta buscar el id de esa cuenta en la tabla Personas:

*  En la secci�n try se intenta recuperar el id mediante un fetchone que se almacena en amigo_id.

*  Si el SELECT falla, el fetchone()[0] falla, y el control se transfiere a la secci�n except.

*  Si se entra en el c�digo except significa que la fila no se ha encontrado en la tabla Personas, por lo que hay que insertarla usando un  INSERT OR IGNORE y luego un commit() para forzar a que la base de datos se actualice.  Si la inserci�n ha tenido �xito se usa cur.lastrowid para averiguar el valor asignado a la columna id en la fila creada.

*  Conocidos los dos id�s se insertan ambos en la tabla de Seguimientos.


Una vez se haya ejecutado varias veces el programa, se habr�n poblado las 2 tablas, y entonces se podr�n consultar para saber las relaciones de amistad. Para ello se va a crear otro programa que mostrar�:

*  La informaci�n de las tablas Personas y Seguimientos

*  La informaci�n compactada de una persona y a las personas que sigue

Para mostrar la informaci�n de las tablas Personas y Seguimientos se consultan las tablas con SELECT:


Para recuperar a las personas que sigue una persona en concreto se realiza un SELECT con una cla�sula JOIN.